In [ ]:
### Importing things for the lab.
import json
import requests
import pandas as pd
from datetime import datetime

### Challenge 1

In [5]:
### Importing from an user page hence /users/.
response = requests.get('https://api.github.com/users/ta-data-lis/repos')
data = pd.DataFrame(response.json())

In [ ]:
### Checking column names
data.head(10)
### Ok... language column is ommited in the ...
column_names = [col for col in data.columns]
### There it is...
column_names

In [18]:
fork_languages = list(data['language'])
### Displaying its contents on a list.
fork_languages

### Challenge 2

In [19]:
### Importing from an repo page hence /repos/.
response = requests.get('https://api.github.com/repos/ta-data-lis/lab-mysql-first-queries/commits')
data = pd.DataFrame(response.json())
# Finding the date.
data.head(10)
# Given that commit has a lot of different elements that can be flattened I suspect it is there.

,sha,node_id,commit,url,html_url,comments_url,author,committer,parents
0,6a243303cfd846faac16417cad54cd40ab3f82c6,MDY6Q29tbWl0MjAzMTMyMjg2OjZhMjQzMzAzY2ZkODQ2Zm...,"{'author': {'name': 'Carlos Madriz', 'email': ...",https://api.github.com/repos/ta-data-lis/lab-m...,https://github.com/ta-data-lis/lab-mysql-first...,https://api.github.com/repos/ta-data-lis/lab-m...,"{'login': 'carlosmd14', 'id': 45972288, 'node_...","{'login': 'web-flow', 'id': 19864447, 'node_id...",[{'sha': 'ee6d046595a26c76425df9fffdb6ed6737c7...
1,ee6d046595a26c76425df9fffdb6ed6737c7c06b,MDY6Q29tbWl0MjAzMTMyMjg2OmVlNmQwNDY1OTVhMjZjNz...,"{'author': {'name': 'Carlos Madriz', 'email': ...",https://api.github.com/repos/ta-data-lis/lab-m...,https://github.com/ta-data-lis/lab-mysql-first...,https://api.github.com/repos/ta-data-lis/lab-m...,"{'login': 'carlosmd14', 'id': 45972288, 'node_...","{'login': 'web-flow', 'id': 19864447, 'node_id...",[{'sha': '381994f755b6b95ffc4fe512f36e2640bd04...
2,381994f755b6b95ffc4fe512f36e2640bd04034f,MDY6Q29tbWl0MjAzMTMyMjg2OjM4MTk5NGY3NTViNmI5NW...,"{'author': {'name': 'Carlos Madriz', 'email': ...",https://api.github.com/repos/ta-data-lis/lab-m...,https://github.com/ta-data-lis/lab-mysql-first...,https://api.github.com/repos/ta-data-lis/lab-m...,"{'login': 'carlosmd14', 'id': 45972288, 'node_...","{'login': 'web-flow', 'id': 19864447, 'node_id...",[{'sha': '2e9b232b72bec7218cd77f171098a7391d89...
3,2e9b232b72bec7218cd77f171098a7391d89ba8c,MDY6Q29tbWl0MjAzMTMyMjg2OjJlOWIyMzJiNzJiZWM3Mj...,"{'author': {'name': 'Carla Mendes', 'email': '...",https://api.github.com/repos/ta-data-lis/lab-m...,https://github.com/ta-data-lis/lab-mysql-first...,https://api.github.com/repos/ta-data-lis/lab-m...,"{'login': 'carlarsmendes', 'id': 38359702, 'no...","{'login': 'carlarsmendes', 'id': 38359702, 'no...",[]


In [20]:
def flatten(data, col_list):
    for column in col_list:
        flattened = pd.DataFrame(dict(data[column])).transpose()
        columns = [str(col) for col in flattened.columns]
        flattened.columns = [column + '_' + colname for colname in columns]
        data = pd.concat([data, flattened], axis=1)
        data = data.drop(column, axis=1)
    return data

In [21]:
# And it is...
data = flatten(data,['commit'])
# But needs to be flattened again...
data = flatten(data,['commit_committer'])
# we got... commit_committer_date

In [22]:
# Making some helpful functions
def dealing_with_dates(my_list):
    my_output_list = []
    for i in my_list:
        my_output_list.append(datetime.strptime(i, "%Y-%m-%dT%H:%M:%SZ"))
    return my_output_list

def days_hours_minutes(td):
    return td.days, td.seconds//3600, (td.seconds//60)%60

In [23]:
# counting.
dates = list(data['commit_committer_date'])
dates_as_date = dealing_with_dates(dates)

my_now = datetime.now().strftime("%Y-%m-%dT%H:%M:%SZ")
my_now = datetime.strptime(my_now, "%Y-%m-%dT%H:%M:%SZ")

dates_timesdeltas_tonow = [my_now - date for date in dates_as_date]
dayspassed = [timedelta.days for timedelta in dates_timesdeltas_tonow]
count_if_less_two_weeks = sum(myday < 14 for myday in dayspassed)
count_if_less_two_weeks

0